# Building and Refining Models

This notebook shows a variety of attempts at finding the best model to count people in drone footage.

TensorFlow wasn't loading properly on my Windows computer, so I followed this link to create a new environment to run tensorflow things: https://medium.com/@mengjiunchiou/how-to-set-keras-with-tensorflow-with-conda-virtual-environment-on-ubuntu-5d56d22e3dc7

In [8]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [9]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io, filters, measure
from scipy import ndimage
from keras.models import Sequential
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from PIL import Image # for conversion to grayscale

In [3]:
# Data have been pre-processed already. The raw images were converted
# to grayscale and were split into 25 smaller images. Labeled images
# were created from the original images and are a series of 0 (no 
# person) and 255 (a person). The labeled images were also split
# into 25 pieces so that they would match the original images. 

# Find location of image files and labeled images
data = glob('data/raw/resized/with_people/splits/*hsv*.png')
labels = glob('data/processed/dots/with_people/splits/*.png')

# Split into the training and testing data
train_X, test_X = train_test_split(data, test_size=0.2, random_state=33)
train_Y, test_Y = train_test_split(labels, test_size=0.2, random_state=33)

In [4]:
# Create function to make a data generator for the model
def gather_images(images, labels, batch_size=10): 
    """ Takes the original and labeled images, combines them into np """
    """ arrays, and passes to model"""
    while 1: 
        for offset in range(0, len(images), batch_size): 
            X = [] # empty list for training data
            Y = [] # empty list for labels 
            for img in images[offset:offset+batch_size]: # for each image in the list
                img_temp = cv2.imread(img)
                img_flatten = np.array(img_temp)# [:,:,2:3]) # create np array [0:108, 0:192, 0:1]
                X.append(img_flatten) # and add to list for X
            for lab in labels[offset:offset+batch_size]: # for each label in the list
                label_temp = io.imread(lab, as_gray=True)
                labels_temp = measure.label(label_temp)
                label_flatten = labels_temp.max() # create np array
                Y.append(label_flatten) # and add to list for y
            yield (np.array(X), np.array(Y).reshape(len(Y),1)) # yield X and y for the model

# Adding a Normalization Layer

## Using the all channels from HSV, Batch Size = 30, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [6]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (5,5), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 243s 701ms/step - loss: 0.5793 - mean_squared_error: 0.5793 - mean_absolute_error: 0.3062 - val_loss: 0.2695 - val_mean_squared_error: 0.2695 - val_mean_absolute_error: 0.2395

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 213s 614ms/step - loss: 0.2984 - mean_squared_error: 0.2984 - mean_absolute_error: 0.2514 - val_loss: 0.2592 - val_mean_squared_error: 0.2592 - val_mean_absolute_error: 0.2483

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 214s 616ms/step - loss: 0.2815 - mean_squared_error: 0.2815 - mean_absolute_error: 0.2356 - val_loss: 0.2386 - val_mean_squared_error: 0.2386 - val_mean_absolute_error: 0.1980

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 211s 608ms/step - loss: 0.2594 - mean_squared_error: 0.2594 - mean_absolute_error: 0.2136 - v

## Using the all channels from HSV, Batch Size = 30, Normalization layer, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [7]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,3)),
  tf.keras.layers.Conv2D(8, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 310s 893ms/step - loss: 0.2641 - mean_squared_error: 0.2641 - mean_absolute_error: 0.2226 - val_loss: 0.1829 - val_mean_squared_error: 0.1829 - val_mean_absolute_error: 0.1546

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 307s 884ms/step - loss: 0.2110 - mean_squared_error: 0.2110 - mean_absolute_error: 0.1780 - val_loss: 0.1721 - val_mean_squared_error: 0.1721 - val_mean_absolute_error: 0.1539

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 310s 894ms/step - loss: 0.1971 - mean_squared_error: 0.1971 - mean_absolute_error: 0.1639 - val_loss: 0.1961 - val_mean_squared_error: 0.1961 - val_mean_absolute_error: 0.1715

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 305s 879ms/step - loss: 0.1853 - mean_squared_error: 0.1853 - mean_absolute_error: 0.1629 - v

# Using Different HSV Channels

## Using the first channel from HSV, Batch Size = 30, Normalization layer, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [43]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(8, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 215s 618ms/step - loss: 0.3042 - mean_squared_error: 0.3042 - mean_absolute_error: 0.2493 - val_loss: 0.2590 - val_mean_squared_error: 0.2590 - val_mean_absolute_error: 0.2260

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 180s 518ms/step - loss: 0.2792 - mean_squared_error: 0.2792 - mean_absolute_error: 0.2305 - val_loss: 0.2314 - val_mean_squared_error: 0.2314 - val_mean_absolute_error: 0.1905

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 174s 501ms/step - loss: 0.2548 - mean_squared_error: 0.2548 - mean_absolute_error: 0.2143 - val_loss: 0.2368 - val_mean_squared_error: 0.2368 - val_mean_absolute_error: 0.2039

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 174s 500ms/step - loss: 0.2396 - mean_squared_error: 0.2396 - mean_absolute_error: 0.2046 - v

## Using the second channel from HSV, Batch Size = 30, Normalization layer, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [59]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(8, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 178s 512ms/step - loss: 0.2521 - mean_squared_error: 0.2521 - mean_absolute_error: 0.2102 - val_loss: 0.1915 - val_mean_squared_error: 0.1915 - val_mean_absolute_error: 0.1471

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 174s 503ms/step - loss: 0.2053 - mean_squared_error: 0.2053 - mean_absolute_error: 0.1718 - val_loss: 0.1694 - val_mean_squared_error: 0.1694 - val_mean_absolute_error: 0.1365

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 173s 499ms/step - loss: 0.1699 - mean_squared_error: 0.1699 - mean_absolute_error: 0.1492 - val_loss: 0.1529 - val_mean_squared_error: 0.1529 - val_mean_absolute_error: 0.1496

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 180s 520ms/step - loss: 0.1543 - mean_squared_error: 0.1543 - mean_absolute_error: 0.1404 - v

## Using the third channel from HSV, Batch Size = 30, Normalization layer, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [61]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(8, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 174s 500ms/step - loss: 0.3035 - mean_squared_error: 0.3035 - mean_absolute_error: 0.2511 - val_loss: 0.2528 - val_mean_squared_error: 0.2528 - val_mean_absolute_error: 0.1901

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 171s 492ms/step - loss: 0.2514 - mean_squared_error: 0.2514 - mean_absolute_error: 0.2074 - val_loss: 0.2209 - val_mean_squared_error: 0.2209 - val_mean_absolute_error: 0.1547

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 171s 492ms/step - loss: 0.2277 - mean_squared_error: 0.2277 - mean_absolute_error: 0.1831 - val_loss: 0.1946 - val_mean_squared_error: 0.1946 - val_mean_absolute_error: 0.1599

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 171s 492ms/step - loss: 0.2079 - mean_squared_error: 0.2079 - mean_absolute_error: 0.1660 - v

# Changing the Dropout

## Using the first channel from HSV, Batch Size = 30, Normalization layer, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling, Dropout # 1 with 0.5

In [52]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(8, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 176s 506ms/step - loss: 0.3045 - mean_squared_error: 0.3045 - mean_absolute_error: 0.2489 - val_loss: 0.2591 - val_mean_squared_error: 0.2591 - val_mean_absolute_error: 0.2296

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 173s 498ms/step - loss: 0.2748 - mean_squared_error: 0.2748 - mean_absolute_error: 0.2334 - val_loss: 0.2329 - val_mean_squared_error: 0.2329 - val_mean_absolute_error: 0.2058

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 174s 500ms/step - loss: 0.2556 - mean_squared_error: 0.2556 - mean_absolute_error: 0.2195 - val_loss: 0.2333 - val_mean_squared_error: 0.2333 - val_mean_absolute_error: 0.2038

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 175s 503ms/step - loss: 0.2406 - mean_squared_error: 0.2406 - mean_absolute_error: 0.2092 - v

# Changing the Nodes in the Convolutional Layers

## Using the first channel from HSV, Batch Size = 30, Normalization Layer, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling, all Conv layers with 64 nodes

In [48]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(64, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(64, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 911s 3s/step - loss: 0.3076 - mean_squared_error: 0.3076 - mean_absolute_error: 0.2493 - val_loss: 0.2689 - val_mean_squared_error: 0.2689 - val_mean_absolute_error: 0.2449

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 864s 2s/step - loss: 0.2906 - mean_squared_error: 0.2906 - mean_absolute_error: 0.2441 - val_loss: 0.2439 - val_mean_squared_error: 0.2439 - val_mean_absolute_error: 0.2004

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 841s 2s/step - loss: 0.2838 - mean_squared_error: 0.2838 - mean_absolute_error: 0.2356 - val_loss: 0.2620 - val_mean_squared_error: 0.2620 - val_mean_absolute_error: 0.2636

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 835s 2s/step - loss: 0.2900 - mean_squared_error: 0.2900 - mean_absolute_error: 0.2446 - val_loss: 0.2

## Using the first channel from HSV, Batch Size = 30, Normalization Layer, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling, all Conv layers with 16 nodes

In [50]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(16, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 237s 684ms/step - loss: 0.3003 - mean_squared_error: 0.3003 - mean_absolute_error: 0.2492 - val_loss: 0.2421 - val_mean_squared_error: 0.2421 - val_mean_absolute_error: 0.2524

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 233s 672ms/step - loss: 0.2685 - mean_squared_error: 0.2685 - mean_absolute_error: 0.2256 - val_loss: 0.2339 - val_mean_squared_error: 0.2339 - val_mean_absolute_error: 0.1854

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 227s 654ms/step - loss: 0.2507 - mean_squared_error: 0.2507 - mean_absolute_error: 0.2113 - val_loss: 0.2278 - val_mean_squared_error: 0.2278 - val_mean_absolute_error: 0.2203

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 227s 654ms/step - loss: 0.2378 - mean_squared_error: 0.2378 - mean_absolute_error: 0.2053 - v

## Using the first channel from HSV, Batch Size = 30, Normaization layer, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling, all Conv layers with 32 nodes

In [49]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(32, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 428s 1s/step - loss: 0.3025 - mean_squared_error: 0.3025 - mean_absolute_error: 0.2474 - val_loss: 0.2614 - val_mean_squared_error: 0.2614 - val_mean_absolute_error: 0.2247

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 415s 1s/step - loss: 0.2811 - mean_squared_error: 0.2811 - mean_absolute_error: 0.2345 - val_loss: 0.2451 - val_mean_squared_error: 0.2451 - val_mean_absolute_error: 0.1913

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 437s 1s/step - loss: 0.2625 - mean_squared_error: 0.2625 - mean_absolute_error: 0.2200 - val_loss: 0.2305 - val_mean_squared_error: 0.2305 - val_mean_absolute_error: 0.2258

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 445s 1s/step - loss: 0.2397 - mean_squared_error: 0.2397 - mean_absolute_error: 0.2072 - val_loss: 0.2

# Changing Batch Size

## Using all channels from HSV, Batch Size = 50, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling, Conv # 3 = 5x5 with 3x3 pooling

In [6]:
batch_size = 50

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (5,5), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
195/195 [==============================]195/195 [==============================] - 174s 891ms/step - loss: 3.0571 - mean_squared_error: 3.0571 - mean_absolute_error: 0.5316

Epoch 2/10
195/195 [==============================]195/195 [==============================] - 173s 886ms/step - loss: 0.2951 - mean_squared_error: 0.2951 - mean_absolute_error: 0.2496

Epoch 3/10
195/195 [==============================]195/195 [==============================] - 172s 881ms/step - loss: 0.2852 - mean_squared_error: 0.2852 - mean_absolute_error: 0.2394

Epoch 4/10
195/195 [==============================]195/195 [==============================] - 177s 905ms/step - loss: 0.2722 - mean_squared_error: 0.2722 - mean_absolute_error: 0.2314

Epoch 5/10
195/195 [==============================]195/195 [==============================] - 179s 919ms/step - loss: 0.2660 - mean_squared_error: 0.2660 - mean_absolute_error: 0.2262

Epoch 6/10
195/195 [==============================]195/195 [===============

## Using all channels from HSV, Batch Size = 50, Conv # 1 = 5x5 with 3x3 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [7]:
batch_size = 50

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (5,5), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
195/195 [==============================]195/195 [==============================] - 178s 914ms/step - loss: 17.6068 - mean_squared_error: 17.6068 - mean_absolute_error: 0.9950

Epoch 2/10
195/195 [==============================]195/195 [==============================] - 184s 942ms/step - loss: 0.3048 - mean_squared_error: 0.3048 - mean_absolute_error: 0.2523

Epoch 3/10
195/195 [==============================]195/195 [==============================] - 177s 907ms/step - loss: 0.2909 - mean_squared_error: 0.2909 - mean_absolute_error: 0.2500

Epoch 4/10
195/195 [==============================]195/195 [==============================] - 179s 917ms/step - loss: 0.2779 - mean_squared_error: 0.2779 - mean_absolute_error: 0.2404

Epoch 5/10
195/195 [==============================]195/195 [==============================] - 182s 934ms/step - loss: 0.2717 - mean_squared_error: 0.2717 - mean_absolute_error: 0.2283

Epoch 6/10
195/195 [==============================]195/195 [=============

# Changing Conv # 1 and pooling

## Uing all channels from HSV, Batch Size = 30, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [8]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (10,10), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
325/325 [==============================]325/325 [==============================] - 416s 1s/step - loss: 3.5330 - mean_squared_error: 3.5330 - mean_absolute_error: 0.4980

Epoch 2/10
325/325 [==============================]325/325 [==============================] - 425s 1s/step - loss: 0.2964 - mean_squared_error: 0.2964 - mean_absolute_error: 0.2509

Epoch 3/10
325/325 [==============================]325/325 [==============================] - 398s 1s/step - loss: 0.2901 - mean_squared_error: 0.2901 - mean_absolute_error: 0.2491

Epoch 4/10
325/325 [==============================]325/325 [==============================] - 393s 1s/step - loss: 0.2842 - mean_squared_error: 0.2842 - mean_absolute_error: 0.2399

Epoch 5/10
325/325 [==============================]325/325 [==============================] - 410s 1s/step - loss: 0.2755 - mean_squared_error: 0.2755 - mean_absolute_error: 0.2345

Epoch 6/10
325/325 [==============================]325/325 [==============================

## Uing first channel from HSV, Batch Size = 30, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 5x5 with 3x3 pooling

In [46]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(8, (10,10), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 322s 928ms/step - loss: 0.3013 - mean_squared_error: 0.3013 - mean_absolute_error: 0.2477 - val_loss: 0.2552 - val_mean_squared_error: 0.2552 - val_mean_absolute_error: 0.2248

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 330s 950ms/step - loss: 0.2839 - mean_squared_error: 0.2839 - mean_absolute_error: 0.2353 - val_loss: 0.2412 - val_mean_squared_error: 0.2412 - val_mean_absolute_error: 0.1863

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 333s 959ms/step - loss: 0.2662 - mean_squared_error: 0.2662 - mean_absolute_error: 0.2196 - val_loss: 0.2281 - val_mean_squared_error: 0.2281 - val_mean_absolute_error: 0.1767

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 324s 935ms/step - loss: 0.2544 - mean_squared_error: 0.2544 - mean_absolute_error: 0.2128 - v

## Uing first channel from HSV, Batch Size = 30, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 5x5 with 3x3 pooling, all conv layers with 64 nodes

In [47]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(input_shape=(108,192,1)),
  tf.keras.layers.Conv2D(64, (10,10), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(64, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10, 
                    validation_data=gather_images(test_X, test_Y, batch_size=batch_size), 
                    validation_steps = len(test_X)//batch_size)

Epoch 1/10
347/347 [==============================]347/347 [==============================] - 771s 2s/step - loss: 0.3043 - mean_squared_error: 0.3043 - mean_absolute_error: 0.2497 - val_loss: 0.2614 - val_mean_squared_error: 0.2614 - val_mean_absolute_error: 0.2308

Epoch 2/10
347/347 [==============================]347/347 [==============================] - 761s 2s/step - loss: 0.2866 - mean_squared_error: 0.2866 - mean_absolute_error: 0.2411 - val_loss: 0.2598 - val_mean_squared_error: 0.2598 - val_mean_absolute_error: 0.2330

Epoch 3/10
347/347 [==============================]347/347 [==============================] - 761s 2s/step - loss: 0.2773 - mean_squared_error: 0.2773 - mean_absolute_error: 0.2302 - val_loss: 0.2559 - val_mean_squared_error: 0.2559 - val_mean_absolute_error: 0.2468

Epoch 4/10
347/347 [==============================]347/347 [==============================] - 769s 2s/step - loss: 0.2725 - mean_squared_error: 0.2725 - mean_absolute_error: 0.2322 - val_loss: 0.2

# Changing Conv 1 and 2 and pooling

## Uses all channels from HSV, Batch Size = 30, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 10x10 with 5x5 pooling

In [9]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (10,10), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (10,10), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
325/325 [==============================]325/325 [==============================] - 412s 1s/step - loss: 1.7279 - mean_squared_error: 1.7279 - mean_absolute_error: 0.3966

Epoch 2/10
325/325 [==============================]325/325 [==============================] - 406s 1s/step - loss: 0.2950 - mean_squared_error: 0.2950 - mean_absolute_error: 0.2484

Epoch 3/10
325/325 [==============================]325/325 [==============================] - 416s 1s/step - loss: 0.2912 - mean_squared_error: 0.2912 - mean_absolute_error: 0.2432

Epoch 4/10
325/325 [==============================]325/325 [==============================] - 427s 1s/step - loss: 0.2857 - mean_squared_error: 0.2857 - mean_absolute_error: 0.2410

Epoch 5/10
325/325 [==============================]325/325 [==============================] - 423s 1s/step - loss: 0.2765 - mean_squared_error: 0.2765 - mean_absolute_error: 0.2344

Epoch 6/10
325/325 [==============================]325/325 [==============================

## Uses all channels from HSV, Batch Size = 50, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 10x10 with 5x5 pooling

In [10]:
batch_size = 50

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (10,10), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (10,10), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
195/195 [==============================]195/195 [==============================] - 405s 2s/step - loss: 2.9963 - mean_squared_error: 2.9963 - mean_absolute_error: 0.4759

Epoch 2/10
195/195 [==============================]195/195 [==============================] - 400s 2s/step - loss: 0.2990 - mean_squared_error: 0.2990 - mean_absolute_error: 0.2558

Epoch 3/10
195/195 [==============================]195/195 [==============================] - 405s 2s/step - loss: 0.2997 - mean_squared_error: 0.2997 - mean_absolute_error: 0.2579

Epoch 4/10
195/195 [==============================]195/195 [==============================] - 434s 2s/step - loss: 0.2995 - mean_squared_error: 0.2995 - mean_absolute_error: 0.2580

Epoch 5/10
195/195 [==============================]195/195 [==============================] - 433s 2s/step - loss: 0.2990 - mean_squared_error: 0.2990 - mean_absolute_error: 0.2580

Epoch 6/10
195/195 [==============================]195/195 [==============================

## Uses all channels from HSV, Batch Size = 30, Conv # 1 = 10x10 with 5x5 pooling, Conv # 2 = 10x10 with 5x5 pooling, Conv # 3 = 5x5 with 2x2 pooling

In [11]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (10,10), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(16, (10,10), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(5,5)),
  tf.keras.layers.Conv2D(32, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
325/325 [==============================]325/325 [==============================] - 474s 1s/step - loss: 0.5884 - mean_squared_error: 0.5884 - mean_absolute_error: 0.2957

Epoch 2/10
325/325 [==============================]325/325 [==============================] - 426s 1s/step - loss: 0.2975 - mean_squared_error: 0.2975 - mean_absolute_error: 0.2553

Epoch 3/10
325/325 [==============================]325/325 [==============================] - 422s 1s/step - loss: 0.2962 - mean_squared_error: 0.2962 - mean_absolute_error: 0.2539

Epoch 4/10
325/325 [==============================]325/325 [==============================] - 421s 1s/step - loss: 0.2976 - mean_squared_error: 0.2976 - mean_absolute_error: 0.2548

Epoch 5/10
325/325 [==============================]325/325 [==============================] - 438s 1s/step - loss: 0.2969 - mean_squared_error: 0.2969 - mean_absolute_error: 0.2577

Epoch 6/10
325/325 [==============================]325/325 [==============================